# Hyper-Entities Explorer
Multi-column sorting and filtering

In [ ]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_parquet('results/entities_full.parquet')
print(f"Loaded {len(df)} entities")

# Normalize scores to 0-1 for fair comparison
df['concrete_norm'] = df['concrete_score'] / 5
df['dacc_norm'] = df['dacc_total'] / 20
df['s1_momentum_norm'] = df['s1_current_momentum'] / 12
df['s1_transform_norm'] = df['s1_transformative_potential'] / 6
df['s2_power_norm'] = df['s2_transformative_power'] / 25
df['s2_risk_norm'] = df['s2_systemic_risk'] / 25
df['dacc_defensive_norm'] = df['dacc_defensive'] / 5
df['dacc_decent_norm'] = df['dacc_decentralized'] / 5
df['dacc_demo_norm'] = df['dacc_democratic'] / 5

In [ ]:
# =============================================================================
# COMPOSITE SCORES FOR FINDING HIDDEN GEMS
# =============================================================================

# 1. HIDDEN GEM: High quality but under-hyped
#    Concrete + d/acc aligned, but low current momentum (not yet mainstream)
df['hidden_gem'] = (
    df['concrete_norm'] * 0.35 +      # Must be buildable
    df['dacc_norm'] * 0.35 +          # Must be good direction
    (1 - df['s1_momentum_norm']) * 0.30  # Bonus for being under-the-radar
)

# 2. BUILDABLE GOOD: Practical + beneficial
#    Weighted toward things you could actually build that are d/acc aligned
df['buildable_good'] = (
    df['concrete_norm'] * 0.40 +
    df['dacc_norm'] * 0.40 +
    df['s2_power_norm'] * 0.20        # Transformative potential
)

# 3. DEFENSIVE TECH: Protection-focused
#    Technologies that favor defense over offense
df['defensive_tech'] = (
    df['dacc_defensive_norm'] * 0.40 +
    df['concrete_norm'] * 0.30 +
    df['dacc_decent_norm'] * 0.30     # Decentralized = harder to attack
)

# 4. UNDERDOG POTENTIAL: High ceiling, low current attention
#    Transformative potential divided by current hype
df['underdog'] = (
    (df['s1_transform_norm'] + df['concrete_norm'] + df['dacc_norm']) / 
    (df['s1_momentum_norm'] + 0.1)  # +0.1 to avoid division issues
)

# 5. BALANCED QUALITY: All-around strong
#    Good across all dimensions
df['balanced'] = (
    df['concrete_norm'] * 0.25 +
    df['dacc_norm'] * 0.25 +
    df['s1_transform_norm'] * 0.20 +
    df['s2_power_norm'] * 0.15 +
    (1 - df['s2_risk_norm']) * 0.15  # Lower risk is better
)

print("Composite scores created: hidden_gem, buildable_good, defensive_tech, underdog, balanced")

In [ ]:
# TOP 15 HIDDEN GEMS
# High quality (concrete + d/acc) but under-hyped (low momentum)
print("🔮 HIDDEN GEMS: Buildable, beneficial, but not yet mainstream\n")
df.sort_values('hidden_gem', ascending=False)[
    ['name', 'hidden_gem', 'concrete_score', 'dacc_total', 's1_current_momentum', 'concrete_verdict']
].head(15)

In [ ]:
# TOP 15 BUILDABLE GOOD
# Practical to build + beneficial direction
print("🛠️ BUILDABLE GOOD: Practical + aligned with d/acc values\n")
df.sort_values('buildable_good', ascending=False)[
    ['name', 'buildable_good', 'concrete_score', 'dacc_total', 's2_transformative_power', 'core_technologies']
].head(15)

In [ ]:
# TOP 15 DEFENSIVE TECH
# Protection-focused, decentralized
print("🛡️ DEFENSIVE TECH: Protection over offense, decentralized\n")
df.sort_values('defensive_tech', ascending=False)[
    ['name', 'defensive_tech', 'dacc_defensive', 'dacc_decentralized', 'concrete_score', 'core_technologies']
].head(15)

In [ ]:
# TOP 15 UNDERDOGS
# High potential relative to current attention
print("🚀 UNDERDOGS: High potential-to-hype ratio\n")
df.sort_values('underdog', ascending=False)[
    ['name', 'underdog', 's1_transformative_potential', 's1_current_momentum', 'concrete_score', 'dacc_total']
].head(15)

In [ ]:
# CURATED SELECTION: 15 Diverse Hidden Gems
# Take top from each category, remove duplicates, diverse by cluster

def get_diverse_picks(df, n=15):
    """Get diverse picks across composite scores and clusters."""
    picks = set()
    seen_clusters = {}
    
    # Weight toward hidden_gem but sample from all
    for score_col in ['hidden_gem', 'buildable_good', 'defensive_tech', 'underdog']:
        sorted_df = df.sort_values(score_col, ascending=False)
        for _, row in sorted_df.iterrows():
            if len(picks) >= n:
                break
            # Limit to 2 per cluster for diversity
            cluster = row['cluster_name']
            if seen_clusters.get(cluster, 0) < 2 and row['name'] not in picks:
                picks.add(row['name'])
                seen_clusters[cluster] = seen_clusters.get(cluster, 0) + 1
    
    return df[df['name'].isin(picks)].sort_values('hidden_gem', ascending=False)

curated = get_diverse_picks(df, 15)
print(f"🌟 CURATED 15: Diverse hidden gems across categories\n")
curated[['name', 'hidden_gem', 'concrete_score', 'dacc_total', 'cluster_name', 'core_technologies']]

In [ ]:
# DETAILED VIEW: Show concrete_version for transformed entities
print("📋 DETAILED: What these could actually be\n")
for _, row in curated.iterrows():
    print(f"━━━ {row['name']} ━━━")
    print(f"    Scores: Concrete={row['concrete_score']}, d/acc={row['dacc_total']}, Hidden Gem={row['hidden_gem']:.2f}")
    print(f"    Cluster: {row['cluster_name']}")
    if row['core_technologies']:
        print(f"    Tech: {row['core_technologies'][:80]}...")
    if row['concrete_verdict'] == 'transform' and row['concrete_version']:
        print(f"    → {row['concrete_version'][:150]}...")
    print()

# Interface Ideas for Inspiration

## Current Dashboard Improvements
1. **Add composite score dropdowns** - Let users pick "Hidden Gems", "Buildable Good", etc.
2. **Slider filters** - Filter by min concreteness, min d/acc
3. **Cluster exploration** - Click a cluster to see all entities in it
4. **Random inspiration** - "Show me a random high-quality entity" button
5. **Compare mode** - Select 2-3 entities side by side

## New Interface Options
1. **Card Gallery** - Pinterest-style cards with key info, filterable
2. **2D Scatter Plot** - X=concreteness, Y=d/acc, size=transformative power, color=cluster
3. **Radar Charts** - Compare entities across all dimensions
4. **Decision Tree** - "What kind of tech are you looking for?" → guided filtering
5. **Serendipity Feed** - Tinder-like swipe through entities, save favorites

## For the 10-15 Showcase
- Could be a simple single-page with cards
- Each card: Name, 1-line pitch, core technologies, why it's a hidden gem
- Link to full detail in main dashboard